In [20]:
from pathlib import Path
from enum import Enum
import cv2
from cv2 import imwrite
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from time import time
from scipy.optimize import curve_fit
%matplotlib inline
import scipy.stats
from util import parse_fn, is_bg
from scipy import stats
from scipy.stats import skewnorm
from scipy.special import erf
from scipy.special import voigt_profile
import os
from pathlib import Path
import os
import re
import numpy as np

In [9]:
frame_rate = 92.59
shutter_delay = 0.01 # second
interestd_position = 4 # mm
# exposure time is 5ms

In [12]:
def get_needed_files(folder):

    velocity_path_list = []
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in dirs:
            velocity_path_list.append(os.path.join(root, name))
        
    frames_of_different_v_list = [] # the length of the list = the number of velocity we measured
    for item in velocity_path_list:
        p_pathlist = Path(item).glob('**/*.raw')
        p_path_list = []
        for path in p_pathlist:
            p_path_list.append(path)
        frames_of_different_v_list.append(p_path_list)

    velocity_list = []
    for item in velocity_path_list:
        velocity = re.findall("/(\d+)mm", item)
#         velocity = re.findall("\d+\.\d+", item)
        velocity_list.append(velocity)
             
        
        

    Dict_all = dict(zip(list(np.concatenate(velocity_list)), frames_of_different_v_list))


    used_frame_num = []
    for i in [int(t) for t in list(Dict_all.keys())]:
#     for i in [float(t) for t in list(Dict_all.keys())]:
        used_frame_num.append(round((shutter_delay+ interestd_position/i)*frame_rate)-1)

    print(used_frame_num)
 
    all_5mm_files = []
    for i in range(0,len(used_frame_num), 1):
        files=[]
        for file in os.listdir(velocity_path_list[i]):
            if file.endswith("_00{}.raw".format(used_frame_num[i])) or file.endswith("_0{}.raw".format(used_frame_num[i])):
                name = str(os.path.join(velocity_path_list[i], file))
                files.append(name)
        all_5mm_files.append(files)
    
    Dict_5mm = dict(zip(list(np.concatenate(velocity_list)), all_5mm_files))

    return Dict_all, Dict_5mm, velocity_path_list, velocity_list, used_frame_num